# Import Libraries and File

In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import test values

df_test = pd.read_csv('testsetvalues.csv')
df_test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


## Inspect and Clean Data

In [3]:
# Inspect test data column data types

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
id                       14850 non-null int64
amount_tsh               14850 non-null float64
date_recorded            14850 non-null object
funder                   13981 non-null object
gps_height               14850 non-null int64
installer                13973 non-null object
longitude                14850 non-null float64
latitude                 14850 non-null float64
wpt_name                 14850 non-null object
num_private              14850 non-null int64
basin                    14850 non-null object
subvillage               14751 non-null object
region                   14850 non-null object
region_code              14850 non-null int64
district_code            14850 non-null int64
lga                      14850 non-null object
ward                     14850 non-null object
population               14850 non-null int64
public_meeting           14029 non-null object
r

In [4]:
# Check for missing/null values

df_test.isnull().sum()

id                          0
amount_tsh                  0
date_recorded               0
funder                    869
gps_height                  0
installer                 877
longitude                   0
latitude                    0
wpt_name                    0
num_private                 0
basin                       0
subvillage                 99
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting            821
recorded_by                 0
scheme_management         969
scheme_name              7092
permit                    737
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity  

In [5]:
# Engineer 'well_age' column out of 'construction_year' and 'date_recorded'

df_test['year_recorded'] = pd.to_datetime(df_test['date_recorded']).dt.year
print(df_test['construction_year'].mean)
df_test['construction_year'] = df_test['construction_year'].replace({0:1999})
df_test['well_age'] = df_test['year_recorded'] - df_test['construction_year']
df_test['well_age'] = df_test['well_age'].replace({-1:12, -2:12, -3:12, -4:12, -5:12, -7:12})
df_test.drop(columns = ['date_recorded', 'year_recorded', 'construction_year'], axis = 1, inplace = True)

<bound method Series.mean of 0        2012
1        2000
2        2010
3        1987
4        2000
5        1990
6        2007
7        1982
8        1997
9        2003
10       2006
11       2002
12          0
13          0
14       1984
15       1989
16          0
17       1978
18          0
19       1982
20       2010
21       2008
22       2005
23       1970
24          0
25       2008
26       2009
27          0
28          0
29       2002
         ... 
14820    2001
14821    2007
14822    1997
14823       0
14824    2008
14825    1978
14826       0
14827       0
14828       0
14829    2005
14830    1996
14831       0
14832    2001
14833    2003
14834       0
14835       0
14836    2009
14837    2010
14838    1986
14839    2005
14840    2009
14841       0
14842    2009
14843    1995
14844       0
14845    1988
14846    1994
14847    2010
14848    2009
14849    2008
Name: construction_year, Length: 14850, dtype: int64>


In [6]:
df_test['well_age'].value_counts()

12    3509
13    1784
14     842
3      650
1      600
2      594
5      514
4      434
7      405
6      351
11     324
8      280
33     255
15     249
16     237
19     203
23     203
10     192
9      192
27     187
31     186
18     184
28     182
17     164
25     161
21     149
29     147
0      141
35     140
41     133
26     133
37     131
20     130
39     127
38      90
36      86
30      78
22      77
43      70
40      62
24      61
34      49
32      42
42      23
50      21
53      20
45      14
44      11
51       7
48       7
46       7
52       5
47       4
49       3
Name: well_age, dtype: int64

In [7]:
# Deal with null/missing values

df_test.drop(columns = ['scheme_name', 'subvillage', 'public_meeting'], axis = 1, inplace = True);
df_test['funder'].fillna('other', inplace = True)
df_test['installer'].fillna('other', inplace = True)
df_test['scheme_management'].fillna('other', inplace = True)
df_test['permit'].fillna('other', inplace = True)

In [8]:
# Check work on removing null values

df_test.isnull().sum()

id                       0
amount_tsh               0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
recorded_by              0
scheme_management        0
permit                   0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
well_age                 0
dtype: int64

In [9]:
# Drop columns that were determined to be dropped from training data

df_test.drop(columns = ['num_private', 'wpt_name', 'ward', 'recorded_by', 'permit', 
                       'management_group', 'payment_type', 'quality_group',
                       'quantity_group', 'source_type', 'source_class',
                       'waterpoint_type_group', 'extraction_type_group', 'extraction_type'], axis = 1, inplace = True)

In [10]:
#Clean 'Funder' column and reduce values counts

def top_funders(var):
    if var in df_test['funder'].value_counts(normalize = True).head(50).index:
        return var
    else:
        return 'other'
    
df_test['top_funded'] = df_test['funder'].map(top_funders)

In [11]:
# Correct data entry grammatical errors

df_test['funder'].replace('0', 'other', inplace = True)
df_test['top_funded'].replace('Finw', 'Fini Water', inplace = True)
df_test['top_funded'].replace('Germany Republi', 'Germany Republic', inplace = True)
df_test['top_funded'].replace('Private', 'Private Individual', inplace = True)
df_test['top_funded'].replace('Jica', 'Jaica', inplace = True)
df_test['top_funded'].replace('Water', 'Ministry Of Water', inplace = True)
df_test['top_funded'].replace('Ministry of Water', 'Ministry Of Water', inplace = True)
df_test['top_funded'].replace('Rural Water Supply And Sanitat', 'Rural Water Supply And Sanitation', inplace = True)

In [12]:
# Inspect changes to value counts

df_test['top_funded'].value_counts()

other                                5331
Government Of Tanzania               2215
Danida                                793
Hesawa                                580
World Bank                            352
Kkkt                                  336
Rwssp                                 329
World Vision                          316
Ministry Of Water                     294
Private Individual                    293
Unicef                                267
Tasaf                                 259
Dhv                                   236
0                                     203
Dwsp                                  201
District Council                      194
Norad                                 184
Germany Republic                      155
Fini Water                            143
Tcrs                                  133
Jaica                                 131
Hifab                                 127
Netherlands                           122
Dwe                               

In [13]:
# Clean 'Installer' column and reduce value counts

def top_installer(var):
    if var in df_test['installer'].value_counts(normalize = True).head(50).index:
        return var
    else:
        return 'other'
    
df_test['top_installers'] = df_test['installer'].map(top_installer)

In [14]:
# Correct data entry grammatical errors

df_test['top_installers'].replace('0', 'other', inplace = True)
df_test['top_installers'].replace('District council', 'District Council', inplace = True)
df_test['top_installers'].replace('Gover', 'Government', inplace = True)
df_test['top_installers'].replace('Commu', 'Community', inplace = True)
df_test['top_installers'].replace('World vision', 'World Vision', inplace = True)
df_test['top_installers'].replace('HESAWA', 'Hesawa', inplace = True)
df_test['top_installers'].replace('DANID', 'DANIDA', inplace = True)
df_test['top_installers'].replace('Centr', 'Central government', inplace = True)
df_test['top_installers'].replace('Gove', 'Government', inplace = True)
df_test['top_installers'].replace('Distri', 'District Council', inplace = True)
df_test['top_installers'].replace('KKKT _ Konde and DWE', 'KKKT', inplace = True)
df_test['top_installers'].replace('Da', 'DANIDA', inplace = True)
df_test['top_installers'].replace('Magadini-Makiwaru wa', 'Magadini-Makiwaru Water Supply', inplace = True)
df_test['top_installers'].replace('Lawatefuka water sup', 'Lawatefuka Water Supply', inplace = True)
df_test['top_installers'].replace('Handeni Trunk Main(', 'Handeni Trunk Main', inplace = True)

In [15]:
# Inspect changes to value counts

df_test['top_installers'].value_counts()

other                             5583
DWE                               4349
Government                         611
DANIDA                             456
Community                          421
Hesawa                             370
RWE                                292
District Council                   253
KKKT                               222
TCRS                               180
World Vision                       165
CES                                155
Central government                 142
TASAF                              108
WEDECO                              99
LGA                                 93
TWESA                               79
WU                                  76
Dmdd                                69
AMREF                               69
ACRA                                68
Villagers                           63
UNICEF                              63
DW                                  62
SEMA                                62
Kiliwater                

In [16]:
# Drop id because it is unnecessary, drop funder and installer because new features were added with their clean data.

df_test.drop(columns = ['id', 'funder', 'installer'], axis = 1, inplace = True)

In [17]:
# Change data type of district_code and region_code to 'string' in order to be one-hot encoded

df_test['district_code'] = df_test['district_code'].astype('str')
df_test['region_code'] = df_test['region_code'].astype('str')

In [18]:
# Inspect remaining columns for any null values. 

df_test.isnull().sum()

amount_tsh               0
gps_height               0
longitude                0
latitude                 0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
population               0
scheme_management        0
extraction_type_class    0
management               0
payment                  0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
well_age                 0
top_funded               0
top_installers           0
dtype: int64

In [19]:
# Check all column data types to ensure they are prepared to be modeled

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 21 columns):
amount_tsh               14850 non-null float64
gps_height               14850 non-null int64
longitude                14850 non-null float64
latitude                 14850 non-null float64
basin                    14850 non-null object
region                   14850 non-null object
region_code              14850 non-null object
district_code            14850 non-null object
lga                      14850 non-null object
population               14850 non-null int64
scheme_management        14850 non-null object
extraction_type_class    14850 non-null object
management               14850 non-null object
payment                  14850 non-null object
water_quality            14850 non-null object
quantity                 14850 non-null object
source                   14850 non-null object
waterpoint_type          14850 non-null object
well_age                 14850 non-null int6

## Model Preparation 

In [20]:
# Isolate continuous variables 

test_cont = df_test[['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'well_age']]

In [21]:
# Isolate categorical variables 

test_cat = df_test.drop(columns = ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population',
                                        'well_age'], axis = 1)

In [22]:
# One-hot encode categorical variables using pd.get_dummies

test_dummies = pd.get_dummies(test_cat, drop_first = True)
test_dummies.head()

,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Dar es Salaam,region_Dodoma,...,top_installers_TASAF,top_installers_TCRS,top_installers_TWESA,top_installers_UNICEF,top_installers_Villagers,top_installers_WEDECO,top_installers_WU,top_installers_WVT,top_installers_World Vision,top_installers_other
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
# Concatenate the OHE categorical and continuous variables back together 

processed_test = pd.concat([test_dummies, test_cont], axis = 1)
processed_test.head()

,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Dar es Salaam,region_Dodoma,...,top_installers_WU,top_installers_WVT,top_installers_World Vision,top_installers_other,amount_tsh,gps_height,longitude,latitude,population,well_age
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.0,1996,35.290799,-4.059696,321,1
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0.0,1569,36.656709,-3.309214,300,13
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.0,1567,34.767863,-5.004344,500,3
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0.0,267,38.058046,-9.418672,250,26
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,500.0,1260,35.006123,-10.950412,60,13


In [25]:
# Save processed test data for use in modeling

processed_test.to_csv('preprocessed_test_data', index = False)

In [ ]:
# Prepare test preds to be submitted

#submit = X_test.copy()
#submit['status_group'] = test_preds
#submit.reset_index(inlpace = True)
#submit = submit.rename(columns = {'index':'id'})
#submit = submit[['id', 'status_group']]